In [1]:
!pip uninstall geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install osmnet
!pip install matplotlib==3.4.3
!pip install pandana
!pip install folium

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

     |████████████████████████████████| 15.4 MB 36 kB/s 
     |████████████████████████████████| 6.3 MB 38.4 MB/s 
     |████████████████████████████████| 994 kB 56.5 MB/s 
     |████████████████████████████████| 994 kB 7.8 MB/s 
     |████████████████████████████████| 10.3 MB 5.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 1.7 MB 5.4 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,329 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 155013 files and directories cur

In [6]:
import geopandas as gpd
import folium

In [7]:
df = gpd.read_file('/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp')
df.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."


In [8]:
df_espg4326 = df.to_crs(epsg=4326)

In [9]:
pop_2019 = df['Pop_2019'] # Dan so 2019
pop_2017 = df['Pop_2017'] # Dan so 2019
growth = pop_2019 / pop_2017


In [10]:
top10_idx = growth.nlargest(10).index
top10_geometry = df['geometry'][top10_idx]
top10_geometry = top10_geometry.to_crs(epsg=4326)
top10_geometry

16    POLYGON ((106.84015 10.89893, 106.84048 10.897...
23    POLYGON ((106.58361 10.92869, 106.58391 10.928...
18    POLYGON ((106.72315 10.72162, 106.72428 10.721...
22    POLYGON ((106.69433 10.89842, 106.69446 10.897...
17    POLYGON ((106.74241 10.80855, 106.74442 10.808...
13    POLYGON ((106.43739 11.10990, 106.43727 11.109...
4     POLYGON ((106.75137 10.76062, 106.75001 10.758...
15    POLYGON ((106.72733 10.89026, 106.72753 10.890...
14    POLYGON ((106.64275 10.72768, 106.64323 10.727...
12    POLYGON ((106.60037 10.82610, 106.60044 10.826...
Name: geometry, dtype: geometry

In [11]:
district_idx = df['Dist_ID'][top10_idx]
district_name = df['Dist_Name'][top10_idx]
print(district_name)
print(district_idx)

16     District 9
23        Hoc Mon
18         Nha Be
22    District 12
17     District 2
13         Cu Chi
4      District 7
15        Thu Duc
14     Binh Chanh
12       Binh Tan
Name: Dist_Name, dtype: object
16    763
23    784
18    786
22    761
17    769
13    783
4     778
15    762
14    785
12    777
Name: Dist_ID, dtype: object


In [12]:
top10_geometry.plot(figsize=(6, 6))

ImportError: ignored

<Figure size 432x432 with 1 Axes>

In [15]:
click = gpd.read_file('/content/response3.json')


DriverError: ignored

In [ ]:
click = click.to_crs(epsg=4326)
click


In [ ]:
merging = gpd.sjoin(click, df_espg4326, how="left", op="within")

In [ ]:
merging

In [ ]:
district_idx.columns = ['Dist_ID']
district_idx

In [ ]:
merging = merging.merge(district_idx)

In [ ]:
merging.head()

In [ ]:
thuduc = merging[merging['Dist_Name'] == 'District 9']
thuduc

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [ ]:
kmeans = KMeans(n_clusters=20)
a=pd.Series(thuduc['geometry'].apply(lambda p: p.x))
b=pd.Series(thuduc['geometry'].apply(lambda p: p.y))
X=np.column_stack((a,b))
y_kmeans = kmeans.fit_predict(X)
thuduc['cluster'] = y_kmeans
thuduc

In [ ]:
thuduc.groupby(['cluster'])

In [ ]:
max_cluster = thuduc.groupby(['cluster']).size().idxmax()
max_cluster


In [ ]:
thuduc[thuduc['cluster'] == max_cluster]

In [ ]:
m = folium.Map(location=[10.70, 106.94], zoom_start=10, tiles='CartoDB positron')
m

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
max_click = thuduc[thuduc['cluster'] == max_cluster]
max_click

In [ ]:
a=pd.Series(max_click['geometry'].apply(lambda p: p.x))
b=pd.Series(max_click['geometry'].apply(lambda p: p.y))
max_click['lat'] = b
max_click['lon'] = a
max_click

In [ ]:
heat_data = [[row['lat'],row['lon']] for index, row in max_click.iterrows()]
heat_data

In [ ]:
HeatMap(heat_data).add_to(m)
m

In [ ]:
def add_heat(df):
    kmeans = KMeans(n_clusters=20)
    a=pd.Series(df['geometry'].apply(lambda p: p.x))
    b=pd.Series(df['geometry'].apply(lambda p: p.y))
    X=np.column_stack((a,b))
    y_kmeans = kmeans.fit_predict(X)
    df['cluster'] = y_kmeans

    max_cluster = df.groupby(['cluster']).size().idxmax()

    max_click = df[df['cluster'] == max_cluster]
    a=pd.Series(max_click['geometry'].apply(lambda p: p.x))
    b=pd.Series(max_click['geometry'].apply(lambda p: p.y))
    max_click['lat'] = b
    max_click['lon'] = a

    heat_data = [[row['lat'],row['lon']] for index, row in max_click.iterrows()]
    HeatMap(heat_data).add_to(m)

In [ ]:
for name in district_name:
    df = merging[merging['Dist_Name'] == name]
    add_heat(df)

In [ ]:
m